In [24]:
quit()

: 

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import time
import datetime

In [2]:
local = SparkSession\
        .builder\
        .master('local')\
        .appName('twitter_local')\
        .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1')\
        .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.3.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/bigdata/.ivy2/cache
The jars for the packages stored in: /home/bigdata/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6de9f071-8db3-4009-9bec-15bef2d0af20;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 354ms :: artifacts dl 10ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifac

23/03/16 16:13:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
local.sparkContext.getConf().getAll()

[('spark.app.submitTime', '1678950830437'),
 ('spark.master', 'local'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1'),
 ('spark.app.startTime', '1678950830664'),
 

In [6]:
%%time
df = local.read.format('mongo')\
               .option('uri', 'mongodb://192.168.56.101:27017/twitter.sample')\
               .load()

df.createOrReplaceTempView('tweets')

CPU times: user 0 ns, sys: 30.1 ms, total: 30.1 ms
Wall time: 3.9 s


In [7]:
%%time
query ='''
  SELECT data.lang as lang, count(*) count
  FROM tweets
  WHERE data.lang is not NULL
  GROUP BY lang
  ORDER BY count DESC;
'''

local.sql(query).show(5)

+----+------+
|lang| count|
+----+------+
|  en|555836|
|  ja|382862|
|  es|137643|
|  th|126300|
|  ko| 73626|
+----+------+
only showing top 5 rows

CPU times: user 8.25 ms, sys: 20.8 ms, total: 29 ms
Wall time: 27.5 s


In [10]:
%%time
query = """
  SELECT
    DATE_FORMAT(_timestamp, "yyyy-MM-dd HH:mm:ss") as time,
    data.text as text
  FROM tweets
  WHERE data.lang = 'en'
"""

en_tweets = local.sql(query)
en_tweets.show(5)

+-------------------+--------------------+
|               time|                text|
+-------------------+--------------------+
|2023-03-14 16:42:09|@erdincciftci Goo...|
|2023-03-14 16:42:09|@CheapremeBrand A...|
|2023-03-14 16:42:09|RT @Joon_record: ...|
|2023-03-14 16:42:09|RT @TheRavenCrypt...|
|2023-03-14 16:42:09|@gareth_lane He i...|
+-------------------+--------------------+
only showing top 5 rows

CPU times: user 118 µs, sys: 5.67 ms, total: 5.78 ms
Wall time: 495 ms


In [9]:
%%time
def text_split(row):
  for word in row.text.split():
    yield Row(time=row.time, word=word)
  

CPU times: user 2 µs, sys: 3 µs, total: 5 µs
Wall time: 7.39 µs


In [11]:
%%time
en_tweets.rdd.take(1)

[Row(time='2023-03-14 16:42:09', text='@erdincciftci Good morning forever Erdin 🙏🔥')]

In [12]:
%%time
en_tweets.rdd.flatMap(text_split).take(5)

[Row(time='2023-03-14 16:42:09', word='@erdincciftci'),
 Row(time='2023-03-14 16:42:09', word='Good'),
 Row(time='2023-03-14 16:42:09', word='morning'),
 Row(time='2023-03-14 16:42:09', word='forever'),
 Row(time='2023-03-14 16:42:09', word='Erdin')]

In [13]:
%%time
en_tweets.rdd.flatMap(text_split).toDF().show(5)

+-------------------+-------------+
|               time|         word|
+-------------------+-------------+
|2023-03-14 16:42:09|@erdincciftci|
|2023-03-14 16:42:09|         Good|
|2023-03-14 16:42:09|      morning|
|2023-03-14 16:42:09|      forever|
|2023-03-14 16:42:09|        Erdin|
+-------------------+-------------+
only showing top 5 rows



In [14]:
%%time
words = en_tweets.rdd.flatMap(text_split).toDF()
words.createOrReplaceTempView('words')

In [15]:
%%time
query = '''
    SELECT word, count(*) count
    FROM words
    GROUP BY word
    ORDER BY count DESC
'''

local.sql(query).show(5)

+----+------+
|word| count|
+----+------+
|  RT|309527|
| the|198030|
|  to|156797|
|   a|115463|
| and|108143|
+----+------+
only showing top 5 rows



In [16]:
%%time
local.sql('SELECT * FROM words').show(5)

+-------------------+-------------+
|               time|         word|
+-------------------+-------------+
|2023-03-14 16:42:09|@erdincciftci|
|2023-03-14 16:42:09|         Good|
|2023-03-14 16:42:09|      morning|
|2023-03-14 16:42:09|      forever|
|2023-03-14 16:42:09|        Erdin|
+-------------------+-------------+
only showing top 5 rows



In [17]:
%%time
local.table('words').count()

8925156

In [18]:
%%time
query = '''
 SELECT substr(time, 1, 13) time,
 word,
 count(*) count
 FROM words
 GROUP BY time, word
''' 
local.sql(query).count()

CPU times: user 30.1 ms, sys: 8.82 ms, total: 38.9 ms
Wall time: 1min 14s


7348846

In [19]:
%%time
#등록 횟수가 적은 단어의 수 조사

query = '''
    SELECT t.count, count(*) words
    FROM (
    SELECT word, count(*) count FROM words
    GROUP BY word
    ) t
    GROUP BY 1
    ORDER BY 1
'''

local.sql(query).show(5)

+-----+------+
|count| words|
+-----+------+
|    1|754452|
|    2|111321|
|    3| 46674|
|    4| 26440|
|    5| 16792|
+-----+------+
only showing top 5 rows

CPU times: user 19.9 ms, sys: 18.9 ms, total: 38.8 ms
Wall time: 1min 8s


In [20]:
%%time
# 단어를 카테고리로 나누는 디멘전 테이블

query = '''
 SELECT word, count,
     IF(count > 1000, word, concat('COUNT=', count)) category
 FROM (
     SELECT word, count(*) count FROM words
     GROUP BY 1
 ) t
'''

local.sql(query).show(10)

+-------------+-----+---------+
|         word|count| category|
+-------------+-----+---------+
|    @_Osasu__|    1|  COUNT=1|
|     @sweenpi|  654|COUNT=654|
|          few| 2038|      few|
|        still| 6085|    still|
|         some| 7548|     some|
|     Holidays|    3|  COUNT=3|
|@questbookapp|    6|  COUNT=6|
|  transaction|   94| COUNT=94|
|          art| 1115|      art|
|       online|  791|COUNT=791|
+-------------+-----+---------+
only showing top 10 rows

CPU times: user 18.7 ms, sys: 11.2 ms, total: 29.9 ms
Wall time: 1min 7s


In [21]:
%%time
local.sql(query).createOrReplaceTempView('word_category')

CPU times: user 51 µs, sys: 5.6 ms, total: 5.65 ms
Wall time: 100 ms


In [22]:
%%time
# 1시간마다 카테고리별로 그룹화하여 집계
query = '''
 SELECT substr(a.time, 1, 13) time, b.category, count(*) count
 FROM words a
 LEFT JOIN word_category b ON a.word = b.word
 GROUP BY 1, 2
'''

local.sql(query).show(5)
local.sql(query).count()

23/03/16 16:49:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/03/16 16:49:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-------------+---------+-----+
|         time| category|count|
+-------------+---------+-----+
|2023-03-16 12| COUNT=21| 1845|
|2023-03-16 00| COUNT=29| 2187|
|2023-03-16 13| COUNT=62|  800|
|2023-03-15 22|COUNT=143|  741|
|2023-03-16 01|COUNT=109|  314|
+-------------+---------+-----+
only showing top 5 rows



CPU times: user 39.5 ms, sys: 96.1 ms, total: 136 ms
Wall time: 4min 13s


24473

In [23]:
%%time
result = local.sql(query).toPandas()
result.head()

23/03/16 16:56:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


CPU times: user 243 ms, sys: 991 ms, total: 1.23 s
Wall time: 2min 7s


,time,category,count
0,2023-03-16 00,COUNT=29,2164
1,2023-03-16 12,COUNT=21,1850
2,2023-03-16 13,COUNT=62,863
3,2023-03-15 22,COUNT=143,641
4,2023-03-15 20,COUNT=289,143
